# Riak

In [1]:
import riak
from pprintpp import pprint as pp
import json
import pandas as pd

import uuid
import hashlib

## Lectura de información en Pandas

Partimos del dataset normalizado y lo desnormalizamos para guardarlo todo junto en RIAK

In [2]:
df_mov = pd.read_excel("../../data/black.xlsx", sheet_name= "Movimientos")
df_miembros = pd.read_excel("../../data/black.xlsx", sheet_name= "Miembros")
df = pd.merge(df_mov, df_miembros, on = ['id_miembro'], how = 'inner')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 0 to 7623
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   id_miembro          7624 non-null   int64         
 1   fecha               7624 non-null   datetime64[ns]
 2   hora                7624 non-null   int64         
 3   minuto              7624 non-null   int64         
 4   importe             7624 non-null   float64       
 5   comercio            6882 non-null   object        
 6   actividad_completa  7621 non-null   object        
 7   actividad           7621 non-null   object        
 8   nombre              7624 non-null   object        
 9   funcion             7624 non-null   object        
 10  organizacion        6060 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(6)
memory usage: 714.8+ KB


In [3]:
df.head(5)

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,77,2006-04-17,11,40,44.56,FCIA TORRES MU¬OZ,FARMACIAS,SALUD,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
1,77,2010-12-18,14,35,3.65,PK VELAZQUEZ JORGE JUAN,GARAJES Y APARCAMIENTOS,COCHE,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
2,77,2009-04-06,11,56,1000.00,ANTONIO HUESO,"ELECTRODOMESTICOS,EQUIPOS ELECTRICOS",HOGAR,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
3,77,2007-10-19,10,49,102.91,CEDIPSA 17017-1 TRUJILLO,CEPSA,COCHE,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular
4,77,2010-02-11,16,48,68.95,CEDIPSA 17018-1 TRUJILLO,CEPSA,COCHE,Ricardo Romero de Tejada y Picatoste,concejal,Partido Popular


## Riak ...

In [4]:
# connect to database
cliente = riak.RiakClient()
cliente.ping()

True

Utilizamos un bucket para guardar los movimientos en formato JSON y otro para guardar el importe acumulado por miembro.

En el bucket de movimientos utilizamos una clave autogenerada (no vamos a poder localizar los movimientos por clave) y en el bucket con el importe acumulado la clave es el nombre de la persona en formato HASH, para no tener problemas ni con los espacios del nombre ni con los acentos

Se van a crear los siguientes buckets:

<style>
table {float:left}
</style>

|bucket|Clave|Contenido|
|: -|-|-|
|movimientos|Marca temporal|Todos los datos del dataset en formnato JSON y desnormalizados|
|acum_importes|Código HASH de la persona que realiza el movimiento|Mapa con el nombre de la persona y el importe acumulado|

El bucket de movientos tendrá los siguientes índices:

|Nombre del índice|Contenido|¿Que busquedas permite hacer?|
|-|-|-|
|idx_miembro_bin|Nombre de la persona|Localizar movimientos de una persona concreta|

In [5]:
BUCKET_MOVIMIENTOS = 'movimientos'
BUCKET_ACUM_IMPORTES = 'acum_importes'

bucket_mov = cliente.bucket(BUCKET_MOVIMIENTOS)
bucket_acum_importes = cliente.bucket_type('maps').bucket(BUCKET_ACUM_IMPORTES)

Función para eliminar los datos de un BUCKET de Riak

In [6]:
def drop_keys(bucket):
    for keys in bucket.stream_keys():
        for key in keys:
            bucket.delete(key)

In [7]:
# Borramos los datos ..
drop_keys(bucket_mov)
drop_keys(bucket_acum_importes)

## Inserción de información en Riak

Para generar los datos en formato JSON partimos del DataFrame de Pandas y lo exportamos a JSON (en formato String) para luego cargarlo en un diccionario de Python

Observa que cuando se genera el dato en formato JSON, los campos fechas se almacenan como un [TIMESTAMP de unix](http://www.unixtimestamp.com), por lo que habría que volver a convertirlo a fecha según el caso

In [8]:
json_string = df.to_json(orient = 'records')
json_list = json.loads(json_string)

In [9]:
print(len(json_list))

7624


In [10]:
pp(json_list[0])

{
    'actividad': 'SALUD',
    'actividad_completa': 'FARMACIAS',
    'comercio': 'FCIA TORRES MU¬OZ',
    'fecha': 1145232000000,
    'funcion': 'concejal',
    'hora': 11,
    'id_miembro': 77,
    'importe': 44.56,
    'minuto': 40,
    'nombre': 'Ricardo Romero de Tejada y Picatoste',
    'organizacion': 'Partido Popular',
}


Vamos a utilizar el nombre de la persona como clave, por lo que necesitamos convertirla previamente a un código que no lleve ni espacios ni acentos.

Este es precisamente el objetivo de la siguiente función ...

In [11]:
def hash_string(s):
    s_utf8 = s.encode('utf-8')
    return hashlib.md5(s_utf8).hexdigest()

Recorremos el dataset de movimientos, almacenando la información tanto en el bucket de movimientos como en el bucket que acumula los movimientos por cliente.

En este último utilizamos una funcionalidad que nos da Riak para guardar la información tipificada. Utilizamos un mapa que tiene dos elementos: Un registro donde guardamos el nombre del cliente y un contador donde guardamos el importe (sin decimales)

In [12]:
# Bucle de carga de datos ...
for movimiento_json in json_list:
    key = uuid.uuid4().hex
    hash_nombre =  hash_string(movimiento_json["nombre"])
    
    movimiento = bucket_mov.new(key, movimiento_json)
    movimiento.add_index('idx_miembro_bin', hash_nombre)
    movimiento.store()
    
    map_nombre = bucket_acum_importes.new(hash_nombre)
    map_nombre.registers['nombre'].assign(movimiento_json["nombre"])
    map_nombre.counters['importe'].increment(int(round(movimiento_json["importe"] * 100)))
    map_nombre.store()
    

In [13]:
for key in bucket_mov.get_keys()[0:3]:
    data = bucket_mov.get(key).data
    print('%s -> %s' % (key, data))

47bcc2b2d1164bc8b5c486e28cf89316 -> {'id_miembro': 80, 'fecha': 1104796800000, 'hora': 17, 'minuto': 55, 'importe': 59.79, 'comercio': 'ESPASA CALPE-CASA DEL LIBRO', 'actividad_completa': 'LIBRERIAS, PAPELERIAS Y DISCOS', 'actividad': 'LIBRERIA', 'nombre': 'Rubén Cruz Orive', 'funcion': 'concejal', 'organizacion': 'Izquierda Unida'}
479de4813e35469eb06a4a90f689f970 -> {'id_miembro': 42, 'fecha': 1077148800000, 'hora': 15, 'minuto': 18, 'importe': 79.4, 'comercio': 'MARISQUERIA LA PESQUERA', 'actividad_completa': 'RESTAURANTE RESTO NIVEL 2', 'actividad': 'RESTAURANTE', 'nombre': 'José María Fernández del Río', 'funcion': 'concejal', 'organizacion': 'Partido Popular'}
4688a2c9614e4151bcf342020184b2df -> {'id_miembro': 60, 'fecha': 1242259200000, 'hora': 20, 'minuto': 31, 'importe': 19.75, 'comercio': 'TAXI DE MADRID 12822', 'actividad_completa': 'TAXIS', 'actividad': 'COCHE', 'nombre': 'Mercedes Rojo Izquierdo', 'funcion': 'concejal', 'organizacion': 'Partido Popular'}


In [14]:
for key in bucket_acum_importes.get_keys()[0:3]:
    data = bucket_acum_importes.get(key)
    print('%s -> %s' % (key, data))

3d9c00de6972d1207cd19e12295754fc -> {('importe', 'counter'): 273266, ('nombre', 'register'): 'Ignacio Varela Díaz'}
38f92f9231f3b264439d377b347de9de -> {('importe', 'counter'): 250058, ('nombre', 'register'): 'José Carlos Contreras Gómez'}
f1c4b4704daf83ff555340400ec5d2cb -> {('importe', 'counter'): 1409109, ('nombre', 'register'): 'Luis Gabarda Durán'}


# Lectura de información en Riak

En la siguiente función partimos del objeto JSON que nos devuelve RIAK y generamos un DataFrame Pandas.

En el caso de la fecha, lo convertimos desde un UNIX Timestamp al formato correcto para Pandas

In [15]:
def json_to_pandas(rows):
    # Convertimos el objeto JSON en un objeto pandas 
    df = pd.read_json(json.dumps(rows))
    
    if 'fecha' in df.columns:
        # Las fechas están en formato UNIX TIMESTAMP. Las volvemos a convertir a formato Date...
        df = df.assign(fecha = pd.to_datetime(df.fecha, unit = 'ms'))

    return df

### Lectura del bucket de movimientos

Cargamos todos los movimientos de la base de datos y lo guardamos en un objeto Pandas, ya que hay ciertas preguntas que no pueden resolverse directamente por esta base de datos

Recuerda que RIAK permite obtener información de un clave, pero no le es posible devolver la información ordenada ...

In [16]:
rows = []
for keys in bucket_mov.stream_keys():
    for key in keys:
        # print('Key %s' % key )
        rows.append(bucket_mov.get(key).data)
        
# Convertimos el objeto json en un objeto pandas 
df_movimientos = json_to_pandas(rows)

In [17]:
df_movimientos.head()

,id_miembro,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,nombre,funcion,organizacion
0,52,2007-08-31,23,35,52.61,LA VACA ARGENTINA,"CAFETERIAS,SNACKS",RESTAURANTE,Luis Gabarda Durán,directivo,None
1,74,2006-07-09,18,4,88.70,None,TAXIS,COCHE,Ramón Ferraz Ricarte,directivo,None
2,73,2006-10-14,20,51,23.20,FAST GOOD EUROBUILDING-GRUPONH,NH HOTELES,HOTEL,Ramón Espinar Gallego,concejal,PSOE
3,39,2007-10-04,10,50,1160.00,VIVATOURS,AGENCIAS DE VIAJES,VIAJE,José Manuel Fernández Norniella,concejal,Partido Popular
4,1,2007-02-16,9,27,150.00,GOLF SANTANDER S.A,CLUBS DEPORTIVOS,DEPORTE,Alberto Recarte García Andrade,concejal,Partido Popular


### Los 10 movimientos mas caros

Esta consulta no la podemos contestar directamente con la Base de Datos por lo que nos apoyamos en un proceso en el cliente

In [18]:
df_movimientos \
    .sort_values('importe', ascending=False)\
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

,nombre,fecha,actividad_completa,importe
6003,Enrique de la Torre Martínez,2007-11-28,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),12000.00
3684,Ramón Ferraz Ricarte,2007-12-19,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),9424.08
4721,Enrique de la Torre Martínez,2008-10-23,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),9000.00
4191,Enrique de la Torre Martínez,2006-11-29,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),8000.00
1315,Miguel Blesa de la Parra,2006-12-26,EL CORTE INGLES,6397.31
5298,Rafael Spottorno Díaz Caro,2007-11-30,CONFECCION TEXTIL EN GENERAL,6375.00
5888,Carmen Contreras Gómez,2006-08-13,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",6198.02
1982,Matías Amat Roca,2006-08-13,BRITISH AIRWAYS - BRITISH A,5763.10
6018,Juan Manuel Astorqui Portera,2005-07-31,AUTOM.Y MOTOCICLETAS ( VENTAS Y REPARAC),5403.97
6888,Enrique de la Torre Martínez,2004-12-06,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),5390.00


### Lectura a través de un índice

In [19]:
rows = []
keys = bucket_mov.stream_index("idx_miembro_bin", hash_string(u"Ildefonso José Sánchez Barcoj"))
for keys in keys.results:
    for movimiento_key in keys:
        rows.append(bucket_mov.get(movimiento_key).data)
    
df = json_to_pandas(rows)

### Los movimientos de una persona concreta

Para obtener los movimientos de una persona en concreto si podemos utilizar el índice que habíamos creado Ad-hoc, aunque la ordenación se realiza en Pandas

In [20]:
df \
    .sort_values('importe', ascending=False) \
    .filter(['nombre', 'fecha', 'actividad_completa', 'importe']) \
    .head(10)

,nombre,fecha,actividad_completa,importe
173,Ildefonso José Sánchez Barcoj,2009-05-04,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),1500.00
138,Ildefonso José Sánchez Barcoj,2010-07-07,EL CORTE INGLES,1478.07
93,Ildefonso José Sánchez Barcoj,2008-03-21,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",1392.79
165,Ildefonso José Sánchez Barcoj,2005-10-28,LOUIS VUITTON/LOEWE,1375.00
161,Ildefonso José Sánchez Barcoj,2009-09-28,AGENCIAS BANCARIAS(ANTICIPO VENTANILLA),1000.00
253,Ildefonso José Sánchez Barcoj,2012-03-15,IBERIA,940.00
196,Ildefonso José Sánchez Barcoj,2010-02-05,CONFECCION TEXTIL EN GENERAL,930.00
88,Ildefonso José Sánchez Barcoj,2005-12-19,EL CORTE INGLES,731.00
187,Ildefonso José Sánchez Barcoj,2009-03-20,PARADORES NACIONALES,708.11
21,Ildefonso José Sánchez Barcoj,2009-12-07,"HOTELES 4 Y 5 ESTRELLAS,BALNEARIOS,CAMPI",672.44


### Lectura de una información agregada

La información agregada está en un mapa, por lo que tenemos que procesarla y convertirla a formato JSON.

In [21]:
rows = []
for keys in bucket_acum_importes.stream_keys():
    #print(keys)
    for key in keys:
        map_nombre = bucket_acum_importes.get(key)
        rows.append({'nombre' : map_nombre.registers['nombre'].value,
                     "importe" : float(map_nombre.counters['importe'].value) / 100})
        
# Convertimos el objeto json en un objeto pandas 
df = json_to_pandas(rows)

### Las 10 personas que mas han gastado

Fácil si disponemos de un agregado. Utilizamos Pandas para ordenar la información ..,

In [22]:
df.sort_values('importe', ascending=False).head(10)

,nombre,importe
57,Enrique de la Torre Martínez,58826.28
24,José Antonio Moral Santín,52447.07
62,Ramón Ferraz Ricarte,45371.52
44,Miguel Blesa de la Parra,40298.84
20,Ildefonso José Sánchez Barcoj,39437.99
63,Juan Manuel Astorqui Portera,36627.89
36,Francisco Baquero Noriega,35256.67
71,Matías Amat Roca,33434.74
4,Carmen Contreras Gómez,31556.45
28,Estanislao Rodríguez-Ponga Salamanca,29350.65
